<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/nb_templates/circuit_expms_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

This will save files to your local machine if `save_files` is set to True.

# Change Inputs Here

In [54]:
task = "numerals"  # choose: numerals, numwords, months
prompt_types = ['done', 'lost', 'names']
num_samps_per_ptype = 512 #768 512

model_name = "gpt2-small"

save_files = True
run_on_other_tasks = True

# Setup

In [55]:
%pip install git+https://github.com/neelnanda-io/TransformerLens.git

  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-3je_em5u
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-3je_em5u
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 829084a53836c5b8b388aa37a5ffce73b6371712
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [85]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
from torch.utils.data import DataLoader
from torch import Tensor

from jaxtyping import Float, Int
from typing import List, Union, Optional, Callable, Tuple, Dict, Literal, Set
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

import pickle
from google.colab import files

import matplotlib.pyplot as plt
import statistics

from tqdm import tqdm

In [57]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

t.set_grad_enabled(False)

## Load Model

In [59]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-small into HookedTransformer


## Import functions from repo

In [60]:
# !git clone https://github.com/wlg1/seqcont_circ_expms.git
!git clone https://github.com/apartresearch/seqcont_circuits.git

fatal: destination path 'seqcont_circuits' already exists and is not an empty directory.


In [61]:
# %cd /content/seqcont_circ_expms
#  node_ablation
%cd /content/seqcont_circuits/src/iter_node_pruning

/content/seqcont_circuits/src/node_ablation


In [62]:
from dataset import Dataset
from metrics import *
from head_ablation_fns import *
from mlp_ablation_fns import *
from node_ablation_fns import *
from loop_node_ablation_fns import *

In [ ]:
%cd /content/seqcont_circuits/src/iter_edge_pruning

from edge_pruning_fns import *

# Load datasets

In [63]:
prompts_list = []

for i in prompt_types:
    # file_name = f'/content/seqcont_circ_expms/data/{task}/{task}_prompts_{i}.pkl'
    file_name = f'/content/seqcont_circuits/data/{task}/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:num_samps_per_ptype]

len(prompts_list)

Van done in 1. Hat done in 2. Ring done in 3. Desk done in 4. Sun done in
Oil lost in 1. Apple lost in 2. Tree lost in 3. Snow lost in 4. Apple lost in
Marcus born in 1. Victoria born in 2. George born in 3. Brandon born in 4. Jamie born in


1536

In [64]:
pos_dict = {}
for i in range(len(model.tokenizer.tokenize(prompts_list[0]['text']))):
    pos_dict['S'+str(i)] = i

In [65]:
dataset_1 = Dataset(prompts_list, pos_dict, model.tokenizer)

In [66]:
# file_name = f'/content/seqcont_circ_expms/data/{task}/randDS_{task}.pkl'
file_name = f'/content/seqcont_circuits/data/{task}/randDS_{task}.pkl'
with open(file_name, 'rb') as file:
    prompts_list_2 = pickle.load(file)

In [67]:
dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer)

# Path patching fns

## Performance Metrics

In [68]:
import gc

t.cuda.empty_cache()
gc.collect()

8

In [70]:
def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset_1: Dataset = dataset_1, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    corr_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset_1.word_idx["end"], dataset_1.corr_tokenIDs]
    incorr_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset_1.word_idx["end"], dataset_1.incorr_tokenIDs]
    # Find logit difference
    answer_logit_diff = corr_logits - incorr_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

model.reset_hooks(including_permanent=True)

ioi_logits_original = model(dataset_1.toks)
abc_logits_original = model(dataset_2.toks)

ioi_average_logit_diff = logits_to_ave_logit_diff_2(ioi_logits_original).item()
abc_average_logit_diff = logits_to_ave_logit_diff_2(abc_logits_original).item()
orig_score = ioi_average_logit_diff

In [71]:
def ioi_metric_3(
    logits: Float[Tensor, "batch seq d_vocab"],
    clean_logit_diff: float = ioi_average_logit_diff,
    corrupted_logit_diff: float = abc_average_logit_diff,
    dataset_1: Dataset = dataset_1,
) -> float:
    patched_logit_diff = logits_to_ave_logit_diff_2(logits, dataset_1)
    return (patched_logit_diff / clean_logit_diff)

print(f"IOI metric (IOI dataset): {ioi_metric_3(ioi_logits_original):.4f}")
print(f"IOI metric (ABC dataset): {ioi_metric_3(abc_logits_original):.4f}")

IOI metric (IOI dataset): 1.0000
IOI metric (ABC dataset): -0.0116


In [72]:
del(ioi_logits_original)
del(abc_logits_original)

## patching fns

In [73]:
# def patch_head_vectors(
#     orig_head_vector: Float[Tensor, "batch pos head_index d_head"],
#     hook: HookPoint,
#     new_cache: ActivationCache,
#     orig_cache: ActivationCache,
#     head_to_patch: Tuple[int, int],
# ) -> Float[Tensor, "batch pos head_index d_head"]:
#     '''
#     '''
#     orig_head_vector[...] = orig_cache[hook.name][...]
#     if head_to_patch[0] == hook.layer():
#         orig_head_vector[:, :, head_to_patch[1]] = new_cache[hook.name][:, :, head_to_patch[1]]
#     return orig_head_vector

# def patch_head_input(
#     orig_activation: Float[Tensor, "batch pos head_idx d_head"],
#     hook: HookPoint,
#     patched_cache: ActivationCache,
#     head_list: List[Tuple[int, int]],
# ) -> Float[Tensor, "batch pos head_idx d_head"]:
#     '''
#     '''
#     heads_to_patch = [head for layer, head in head_list if layer == hook.layer()]
#     orig_activation[:, :, heads_to_patch] = patched_cache[hook.name][:, :, heads_to_patch]
#     return orig_activation

In [74]:
# def patch_mlp_vectors(
#     orig_MLP_vector: Float[Tensor, "batch pos d_model"],
#     hook: HookPoint,
#     new_cache: ActivationCache,
#     orig_cache: ActivationCache,
#     layer_to_patch: int,
# ) -> Float[Tensor, "batch pos d_model"]:
#     '''
#     '''
#     if layer_to_patch == hook.layer():
#         orig_MLP_vector[:, :, :] = new_cache[hook.name][:, :, :]
#     return orig_MLP_vector

# def patch_mlp_input(
#     orig_activation: Float[Tensor, "batch pos d_model"],
#     hook: HookPoint,
#     patched_cache: ActivationCache,
#     layer_list: List[int],
# ) -> Float[Tensor, "batch pos head_idx d_head"]:
#     '''
#     '''
#     if hook.layer() in layer_list:
#         orig_activation[:, :, :] = patched_cache[hook.name][:, :, :]
#     return orig_activation

# only loop thru sender/mlp nodes of circuit

## head to head

In [78]:
# def circ_path_patch_head_to_heads(
#     circuit: List[Tuple[int, int]],
#     receiver_heads: List[Tuple[int, int]],
#     receiver_input: str,
#     model: HookedTransformer,
#     patching_metric: Callable,
#     new_dataset: Dataset = dataset_2,
#     orig_dataset: Dataset = dataset_1,
#     new_cache: Optional[ActivationCache] = None,
#     orig_cache: Optional[ActivationCache] = None,
# ) -> Float[Tensor, "layer head"]:
#     '''
#     Returns:
#         tensor of scores of shape (max(receiver_layers), model.cfg.n_heads)
#     '''
#     assert receiver_input in ("k", "q", "v", "z")
#     receiver_layers = set(next(zip(*receiver_heads)))  # a set of all layers of receiver heads
#     receiver_hook_names = [utils.get_act_name(receiver_input, layer) for layer in receiver_layers]
#     receiver_hook_names_filter = lambda name: name in receiver_hook_names

#     results = t.zeros(max(receiver_layers), model.cfg.n_heads, device="cuda", dtype=t.float32)

#     ### 1. Get activations ###
#     z_name_filter = lambda name: name.endswith("z")
#     if new_cache is None:
#         _, new_cache = model.run_with_cache(
#             new_dataset.toks,
#             names_filter=z_name_filter,
#             return_type=None
#         )
#     if orig_cache is None:
#         _, orig_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=z_name_filter,
#             return_type=None
#         )

#     senders = [tup for tup in circuit if tup[0] < receiver_heads[0][0]]  # senders are in layer before receivers

#     for (sender_layer, sender_head) in tqdm(senders):
#         ### 2. Frozen Clean Run with sender node patched from Corrupted Run ###
#         hook_fn = partial(
#             patch_head_vectors,
#             new_cache=new_cache,
#             orig_cache=orig_cache,
#             head_to_patch=(sender_layer, sender_head),
#         )
#         model.add_hook(z_name_filter, hook_fn, level=1)

#         _, patched_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=receiver_hook_names_filter,
#             return_type=None
#         )
#         # model.reset_hooks(including_permanent=True)
#         assert set(patched_cache.keys()) == set(receiver_hook_names)

#         ### 3. Clean Run, with patched in received node from step 2 ###
#         hook_fn = partial(
#             patch_head_input,
#             patched_cache=patched_cache,
#             head_list=receiver_heads,
#         )
#         patched_logits = model.run_with_hooks(
#             orig_dataset.toks,
#             fwd_hooks = [(receiver_hook_names_filter, hook_fn)],
#             return_type="logits"
#         )

#         results[sender_layer, sender_head] = patching_metric(patched_logits)

#     return results

## mlp to mlp

In [79]:
# def circ_path_patch_MLPs_to_MLPs(
#     mlp_circuit: List[int],
#     receiver_layers: List[int],
#     model: HookedTransformer,
#     patching_metric: Callable,
#     new_dataset: Dataset = dataset_2,
#     orig_dataset: Dataset = dataset_1,
#     new_cache: Optional[ActivationCache] = None,
#     orig_cache: Optional[ActivationCache] = None,
# ) -> Float[Tensor, "layer head"]:
#     '''
#     Returns:
#         tensor of scores of shape (max(receiver_layers), model.cfg.n_heads)
#     '''
#     # model.reset_hooks()
#     receiver_hook_names = [utils.get_act_name('mlp_out', layer) for layer in receiver_layers]  # modify for mlp_out
#     receiver_hook_names_filter = lambda name: name in receiver_hook_names

#     results = t.zeros(max(receiver_layers), device="cuda", dtype=t.float32)

#     ### 1. Get activations ###
#     z_name_filter = lambda name: name.endswith("mlp_out")

#     if new_cache is None:
#         _, new_cache = model.run_with_cache(
#             new_dataset.toks,
#             names_filter=z_name_filter,
#             return_type=None
#         )
#     if orig_cache is None:
#         _, orig_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=z_name_filter,
#             return_type=None
#         )

#     sender_mlp_list = [L for L in mlp_circuit if L < max(receiver_layers)]
#     for (sender_layer) in sender_mlp_list:
#         ### 2. Frozen Clean Run with sender node patched from Corrupted Run ###
#         hook_fn = partial(
#             patch_mlp_vectors,
#             new_cache=new_cache,
#             orig_cache=orig_cache,
#             layer_to_patch = sender_layer
#         )

#         model.add_hook(z_name_filter, hook_fn, level=1)

#         _, patched_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=receiver_hook_names_filter,
#             return_type=None
#         )
#         # model.reset_hooks(including_permanent=True)
#         # assert set(patched_cache.keys()) == set(receiver_hook_names)

#         ### 3. Clean Run with patched receiver node from step 2 ###
#         hook_fn = partial(
#             patch_mlp_input,
#             patched_cache=patched_cache,
#             layer_list=receiver_layers,
#         )
#         patched_logits = model.run_with_hooks(
#             orig_dataset.toks,
#             fwd_hooks = [(receiver_hook_names_filter, hook_fn)],
#             return_type="logits"
#         )

#         results[sender_layer] = patching_metric(patched_logits)

#     # the result is which sender layers affect ALL the inputted nodes. this is why we just
#     # want to pass one node at a time- to see which layers affect just IT.
#     # if we want a 'group of nodes under a common type', we'd pass a set of nodes
#     return results

## head to MLP

head senders to MLP receiver (circ nodes)

In [80]:
# def circ_path_patch_head_to_mlp(
#     circuit: List[Tuple[int, int]],
#     receiver_layers: List[int],
#     model: HookedTransformer,
#     patching_metric: Callable,
#     new_dataset: Dataset = dataset_2,
#     orig_dataset: Dataset = dataset_1,
#     new_cache: Optional[ActivationCache] = None,
#     orig_cache: Optional[ActivationCache] = None,
# ) -> Float[Tensor, "layer head"]:
#     '''
#     Returns:
#         tensor of scores of shape (max(receiver_layers), model.cfg.n_heads)
#     '''
#     # model.reset_hooks()

#     receiver_hook_names = [utils.get_act_name('mlp_out', layer) for layer in receiver_layers]
#     receiver_hook_names_filter = lambda name: name in receiver_hook_names

#     results = t.zeros(max(receiver_layers), model.cfg.n_heads, device="cuda", dtype=t.float32)

#     ### 1. Get activations ###
#     z_name_filter = lambda name: name.endswith(("z", "mlpout"))  # gets same value as just z

#     if new_cache is None:
#         _, new_cache = model.run_with_cache(
#             new_dataset.toks,
#             names_filter=z_name_filter,
#             return_type=None
#         )
#     if orig_cache is None:
#         _, orig_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=z_name_filter,
#             return_type=None
#         )

#     senders = [tup for tup in circuit if tup[0] < receiver_layers[0]]
#     for (sender_layer, sender_head) in tqdm(senders):
#         ### 2. Frozen Clean Run with sender node patched from Corrupted Run ###
#         hook_fn = partial(
#             patch_head_vectors,
#             new_cache=new_cache,
#             orig_cache=orig_cache,
#             head_to_patch=(sender_layer, sender_head),
#         )

#         model.add_hook(z_name_filter, hook_fn, level=1)

#         _, patched_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=receiver_hook_names_filter,
#             return_type=None
#         )
#         # model.reset_hooks(including_permanent=True)
#         # assert set(patched_cache.keys()) == set(receiver_hook_names)

#         ### 3. Clean Run with patched receiver node from step 2 ###

#         hook_fn = partial(
#             patch_mlp_input,
#             patched_cache=patched_cache,
#             layer_list=receiver_layers,
#         )
#         patched_logits = model.run_with_hooks(
#             orig_dataset.toks,
#             fwd_hooks = [(receiver_hook_names_filter, hook_fn)],
#             return_type="logits"
#         )

#         results[sender_layer, sender_head] = patching_metric(patched_logits)

#     return results

## MLP to head

MLP senders to head receiver (circ nodes)

In [81]:
# def circ_path_patch_mlp_to_head(
#     mlp_circuit: List[int],
#     receiver_heads: List[Tuple[int, int]],
#     receiver_input: str,
#     model: HookedTransformer,
#     patching_metric: Callable,
#     new_dataset: Dataset = dataset_2,
#     orig_dataset: Dataset = dataset_1,
#     new_cache: Optional[ActivationCache] = None,
#     orig_cache: Optional[ActivationCache] = None,
# ) -> Float[Tensor, "layer head"]:
#     '''
#     Returns:
#         tensor of scores of shape (max(receiver_layers), model.cfg.n_heads)
#     '''
#     # model.reset_hooks() # doesn't make diff if comment out or not

#     assert receiver_input in ("k", "q", "v", "z")  # we can run get_path_patch_head_to_heads() 3 times for k, q, v!
#     receiver_layers = set(next(zip(*receiver_heads)))
#     receiver_hook_names = [utils.get_act_name(receiver_input, layer) for layer in receiver_layers]
#     receiver_hook_names_filter = lambda name: name in receiver_hook_names

#     results = t.zeros(max(receiver_layers), device="cuda", dtype=t.float32)

#     ### 1. Get activations ###
#     z_name_filter = lambda name: name.endswith(("z", "mlpout"))  # gets same value as just mlp out

#     if new_cache is None:
#         _, new_cache = model.run_with_cache(
#             new_dataset.toks,
#             names_filter=z_name_filter,
#             return_type=None
#         )
#     if orig_cache is None:
#         _, orig_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=z_name_filter,
#             return_type=None
#         )

#     sender_mlp_list = [L for L in mlp_circuit if L < receiver_heads[0][0]]
#     for (sender_layer) in sender_mlp_list:
#         ### 2. Frozen Clean Run with sender node patched from Corrupted Run ###
#         hook_fn = partial(
#             patch_mlp_vectors,
#             new_cache=new_cache,
#             orig_cache=orig_cache,
#             layer_to_patch = sender_layer
#         )

#         model.add_hook(z_name_filter, hook_fn, level=1)

#         _, patched_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=receiver_hook_names_filter,
#             return_type=None
#         )
#         # model.reset_hooks(including_permanent=True)
#         assert set(patched_cache.keys()) == set(receiver_hook_names)

#         ### 3. Clean Run with patched receiver node from step 2 ###
#         hook_fn = partial(
#             patch_head_input,
#             patched_cache=patched_cache,
#             head_list=receiver_heads,
#         )
#         patched_logits = model.run_with_hooks(
#             orig_dataset.toks,
#             fwd_hooks = [(receiver_hook_names_filter, hook_fn)],
#             return_type="logits"
#         )

#         results[sender_layer] = patching_metric(patched_logits)

#     return results

# loop backw

## get circuit

Get the node ablated circuit without edges ablated.

In [83]:
heads_not_ablate = [(0, 1), (1, 5), (4, 4), (4, 10), (5, 0), (6, 1), (6, 6), (6, 10), (7, 11), (8, 1), (8, 6), (8, 8), (8, 9), (9, 1)]
mlps_not_ablate = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]

orig_score = ioi_average_logit_diff

model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
abl_model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

new_logits = model(dataset_1.toks)
new_score = logits_to_ave_logit_diff_2(new_logits, dataset_1)
circ_score = (100 * new_score / orig_score).item()
print(f"(cand circuit / full) %: {circ_score:.4f}")
del(new_logits)

(cand circuit / full) %: 81.0075


## head to head

In [86]:
qkv_to_HH = {} # qkv to dict

for head_type in ["q", "k", "v"]:
    head_to_head_results = {}
    for head in heads_not_ablate:
        print(head_type, head)
        model.reset_hooks()
        model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

        result = circ_path_patch_head_to_heads(
            circuit = heads_not_ablate,
            receiver_heads = [head],
            receiver_input = head_type,
            model = model,
            patching_metric = ioi_metric_3
        )
        head_to_head_results[head] = result
    qkv_to_HH[head_type] = head_to_head_results

q (0, 1)


0it [00:00, ?it/s]


q (1, 5)


100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


q (4, 4)


...

100%|██████████| 9/9 [00:10<00:00,  1.13s/it]


v (8, 9)


100%|██████████| 9/9 [00:10<00:00,  1.13s/it]


v (9, 1)


100%|██████████| 13/13 [00:14<00:00,  1.13s/it]


In [87]:
head_to_head_adjList = {}
for head_type in ["q", "k", "v"]:
    for head in heads_not_ablate:
        result = qkv_to_HH[head_type][head]
        filtered_indices = (result < 0.8) & (result != 0.0)
        rows, cols = filtered_indices.nonzero(as_tuple=True)
        sender_nodes = list(zip(rows.tolist(), cols.tolist()))
        head_with_type = head + (head_type,)
        head_to_head_adjList[head_with_type] = sender_nodes

## mlp to mlp

In [91]:
mlp_to_mlp_results = {}

# for layer in range(11, 0, -1):
for layer in reversed(mlps_not_ablate):
    print(layer)
    model.reset_hooks()
    model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)
    result = circ_path_patch_MLPs_to_MLPs(
        mlp_circuit = mlps_not_ablate,
        receiver_layers = [layer],
        model = model,
        patching_metric = ioi_metric_3
    )
    mlp_to_mlp_results[layer] = result

11
10
9
8
7
6
4
3
2
1
0


In [92]:
mlp_to_mlp_adjList = {}
for mlp in mlps_not_ablate:
    result = mlp_to_mlp_results[mlp]
    filtered_indices = (result < 0.80) & (result != 0.0)
    filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
    mlp_to_mlp_adjList[mlp] = filtered_indices.tolist()

## head to mlp

In [93]:
head_to_mlp_results = {}

# for layer in range(11, 0, -1):
for layer in reversed(mlps_not_ablate):
    print(layer)
    model.reset_hooks()
    model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)
    result = circ_path_patch_head_to_mlp(
        circuit = heads_not_ablate,
        receiver_layers = [layer],
        model = model,
        patching_metric = ioi_metric_3
    )
    head_to_mlp_results[layer] = result

11


100%|██████████| 14/14 [00:15<00:00,  1.13s/it]


10


100%|██████████| 14/14 [00:15<00:00,  1.13s/it]


9


...

100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


1


100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


0


0it [00:00, ?it/s]


In [94]:
head_to_mlp_adjList = {}
for layer in mlps_not_ablate:
    result = head_to_mlp_results[layer]
    filtered_indices = (result < 0.8) & (result != 0.0)
    rows, cols = filtered_indices.nonzero(as_tuple=True)
    sender_nodes = list(zip(rows.tolist(), cols.tolist()))
    head_to_mlp_adjList[layer] = sender_nodes

## mlp to head

In [95]:
qkv_mlp_to_HH = {} # qkv to dict

for head_type in ["q", "k", "v"]:
    mlp_to_head_results = {}
    for head in heads_not_ablate:
        print(head_type, head)
        model.reset_hooks()
        model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

        result = circ_path_patch_mlp_to_head(
            mlp_circuit = mlps_not_ablate,
            receiver_heads = [head],
            receiver_input = head_type,
            model = model,
            patching_metric = ioi_metric_3
        )
        mlp_to_head_results[head] = result
    qkv_mlp_to_HH[head_type] = mlp_to_head_results

q (0, 1)
q (1, 5)
q (4, 4)
q (4, 10)
q (5, 0)
q (6, 1)
q (6, 6)
q (6, 10)
q (7, 11)
q (8, 1)
q (8, 6)
q (8, 8)
q (8, 9)
q (9, 1)
k (0, 1)
k (1, 5)
k (4, 4)
k (4, 10)
k (5, 0)
k (6, 1)
k (6, 6)
k (6, 10)
k (7, 11)
k (8, 1)
k (8, 6)
k (8, 8)
k (8, 9)
k (9, 1)
v (0, 1)
v (1, 5)
v (4, 4)
v (4, 10)
v (5, 0)
v (6, 1)
v (6, 6)
v (6, 10)
v (7, 11)
v (8, 1)
v (8, 6)
v (8, 8)
v (8, 9)
v (9, 1)


In [96]:
mlp_to_head_adjList = {}
for head_type in ["q", "k", "v"]:
    for head in heads_not_ablate:
        result = qkv_mlp_to_HH[head_type][head]
        filtered_indices = (result < 0.8) & (result != 0.0)
        filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
        head_with_type = head + (head_type,)
        mlp_to_head_adjList[head_with_type] = filtered_indices.tolist()

# save graph files to free up memory

In [98]:
with open(task + "_head_to_head_results.pkl", "wb") as file:
    pickle.dump(head_to_head_results, file)
files.download(task + "_head_to_head_results.pkl")

with open(task + "_mlp_to_mlp_results.pkl", "wb") as file:
    pickle.dump(mlp_to_mlp_results, file)
files.download(task + "_mlp_to_mlp_results.pkl")

with open(task + "_head_to_mlp_results.pkl", "wb") as file:
    pickle.dump(head_to_mlp_results, file)
files.download(task + "_head_to_mlp_results.pkl")

with open(task + "_mlp_to_head_results.pkl", "wb") as file:
    pickle.dump(mlp_to_head_results, file)
files.download(task + "_mlp_to_head_results.pkl")

del(head_to_head_results)
del(mlp_to_mlp_results)
del(head_to_mlp_results)
del(mlp_to_head_results)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# resid post

## head to resid

In [99]:
# def get_path_patch_head_to_final_resid_post(
#     circuit: List[Tuple[int, int]],
#     model: HookedTransformer,
#     patching_metric: Callable,
#     new_dataset: Dataset = dataset_2,
#     orig_dataset: Dataset = dataset_1,
# ) -> Float[Tensor, "layer head"]:
#     '''
#     Returns:

#     '''
#     # model.reset_hooks()
#     results = t.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=t.float32)

#     resid_post_hook_name = utils.get_act_name("resid_post", model.cfg.n_layers - 1)
#     resid_post_name_filter = lambda name: name == resid_post_hook_name


#     ### 1. Get activations ###
#     z_name_filter = lambda name: name.endswith("z")

#     _, new_cache = model.run_with_cache(
#         new_dataset.toks,
#         names_filter=z_name_filter,
#         return_type=None
#     )

#     _, orig_cache = model.run_with_cache(
#         orig_dataset.toks,
#         names_filter=z_name_filter,
#         return_type=None
#     )

#     for (sender_layer, sender_head) in tqdm(circuit):
#         ### 2. Frozen Clean Run with sender node patched from Corrupted Run ###
#         hook_fn = partial(
#             patch_head_vectors,
#             new_cache=new_cache,
#             orig_cache=orig_cache,
#             head_to_patch=(sender_layer, sender_head),
#         )
#         model.add_hook(z_name_filter, hook_fn)

#         _, patched_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=resid_post_name_filter,
#             return_type=None
#         )

#         assert set(patched_cache.keys()) == {resid_post_hook_name}

#         ### 3. Clean Run with patched receiver node from step 2 ###
#         patched_logits = model.unembed(model.ln_final(patched_cache[resid_post_hook_name]))

#         results[sender_layer, sender_head] = patching_metric(patched_logits)

#     return results

In [100]:
model.reset_hooks()
model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

path_patch_head_to_final_resid_post = get_path_patch_head_to_final_resid_post(heads_not_ablate, model, ioi_metric_3)

100%|██████████| 14/14 [00:09<00:00,  1.52it/s]


In [101]:
path_patch_head_to_final_resid_post.size()

torch.Size([12, 12])

In [102]:
heads_to_resid = {}
result = path_patch_head_to_final_resid_post
filtered_indices = (result < 0.8) & (result != 0.0)
rows, cols = filtered_indices.nonzero(as_tuple=True)
heads_to_resid['resid'] = list(zip(rows.tolist(), cols.tolist()))

## mlp to resid

In [103]:
# def get_path_patch_mlp_to_final_resid_post(
#     mlp_circuit: List[int],
#     model: HookedTransformer,
#     patching_metric: Callable,
#     new_dataset: Dataset = dataset_2,
#     orig_dataset: Dataset = dataset_1,
# ) -> Float[Tensor, "layer head"]:
#     '''
#     Performs path patching (see algorithm in appendix B of IOI paper), with:

#         sender head = (each head, looped through, one at a time)
#         receiver node = final value of residual stream

#     Returns:
#         tensor of metric values for every possible sender head
#     '''
#     # model.reset_hooks()
#     results = t.zeros(model.cfg.n_layers, device="cuda", dtype=t.float32) #model.cfg.n_heads,

#     resid_post_hook_name = utils.get_act_name("resid_post", model.cfg.n_layers - 1)
#     resid_post_name_filter = lambda name: name == resid_post_hook_name


#     ### 1. Get activations ###
#     z_name_filter = lambda name: name.endswith(("z", "mlp_out"))

#     _, new_cache = model.run_with_cache(
#         new_dataset.toks,
#         names_filter=z_name_filter,
#         return_type=None
#     )

#     _, orig_cache = model.run_with_cache(
#         orig_dataset.toks,
#         names_filter=z_name_filter,
#         return_type=None
#     )

#     for sender_layer in mlp_circuit:
#         ### 2. Frozen Clean Run with sender node patched from Corrupted Run ###
#         hook_fn = partial(
#             patch_mlp_vectors,
#             new_cache=new_cache,
#             orig_cache=orig_cache,
#             layer_to_patch = sender_layer
#         )
#         model.add_hook(z_name_filter, hook_fn)

#         _, patched_cache = model.run_with_cache(
#             orig_dataset.toks,
#             names_filter=resid_post_name_filter,
#             return_type=None
#         )

#         assert set(patched_cache.keys()) == {resid_post_hook_name}

#         ### 3. Clean Run with patched receiver node from step 2 ###

#         patched_logits = model.unembed(model.ln_final(patched_cache[resid_post_hook_name]))

#         results[sender_layer] = patching_metric(patched_logits)

#     return results

In [104]:
model.reset_hooks()
model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

path_patch_mlp_to_final_resid_post = get_path_patch_mlp_to_final_resid_post(mlps_not_ablate, model, ioi_metric_3)

In [105]:
path_patch_mlp_to_final_resid_post.size()

torch.Size([12])

In [106]:
mlps_to_resid = {}
result = path_patch_mlp_to_final_resid_post
filtered_indices = (result < 0.8) & (result != 0.0)
filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
mlps_to_resid['resid'] = filtered_indices.tolist()

# filter out nodes with no ingoing edges

In [107]:
head_to_head_adjList = {node: neighbors for node, neighbors in head_to_head_adjList.items() if neighbors}

In [108]:
mlp_to_head_adjList = {node: neighbors for node, neighbors in mlp_to_head_adjList.items() if neighbors}

# save graph files

In [109]:
with open(task + "_head_to_head_adjList.pkl", "wb") as file:
    pickle.dump(head_to_head_adjList, file)
files.download(task + "_head_to_head_adjList.pkl")

with open(task + "_mlp_to_mlp_adjList.pkl", "wb") as file:
    pickle.dump(mlp_to_mlp_adjList, file)
files.download(task + "_mlp_to_mlp_adjList.pkl")

with open(task + "_head_to_mlp_adjList.pkl", "wb") as file:
    pickle.dump(head_to_mlp_adjList, file)
files.download(task + "_head_to_mlp_adjList.pkl")

with open(task + "_mlp_to_head_adjList.pkl", "wb") as file:
    pickle.dump(mlp_to_head_adjList, file)
files.download(task + "_mlp_to_head_adjList.pkl")

with open(task + "_heads_to_resid.pkl", "wb") as file:
    pickle.dump(heads_to_resid, file)
files.download(task + "_heads_to_resid.pkl")

with open(task + "_mlps_to_resid.pkl", "wb") as file:
    pickle.dump(mlps_to_resid, file)
files.download(task + "_mlps_to_resid.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [110]:
with open(task + "_heads_to_resid_results.pkl", "wb") as file:
    pickle.dump(path_patch_head_to_final_resid_post, file)
files.download(task + "_heads_to_resid_results.pkl")

with open(task + "_mlps_to_resid_results.pkl", "wb") as file:
    pickle.dump(path_patch_mlp_to_final_resid_post, file)
files.download(task + "_mlps_to_resid_results.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# graph plot

## plot qkv

In [112]:
# with open(task + "_head_to_head_adjList.pkl", "rb") as file:
#     head_to_head_adjList = pickle.load(file)

# with open(task + "_mlp_to_mlp_adjList.pkl", "rb") as file:
#     mlp_to_mlp_adjList = pickle.load(file)

# with open(task + "_head_to_mlp_adjList.pkl", "rb") as file:
#     head_to_mlp_adjList = pickle.load(file)

# with open(task + "_mlp_to_head_adjList.pkl", "rb") as file:
#     mlp_to_head_adjList = pickle.load(file)

# with open(task + "_heads_to_resid.pkl", "rb") as file:
#     heads_to_resid = pickle.load(file)

# with open(task + "_mlps_to_resid.pkl", "rb") as file:
#     mlps_to_resid = pickle.load(file)

In [113]:
from graphviz import Digraph, Source
from IPython.display import display
from google.colab import files

def plot_graph_adjacency_qkv(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                             mlp_to_head_adjList, heads_to_resid, mlps_to_resid,
                             filename="circuit_graph", highlighted_nodes=None):
    dot = Digraph()
    dot.attr(ranksep='0.45', nodesep='0.11')  # vert height- ranksep, nodesep- w

    dot.node('resid_post', color="#ffcccb", style='filled')

    for node in mlp_to_mlp_adjList.keys():
        sender_name = "MLP " + str(node)
        dot.node(sender_name, color="#ffcccb", style='filled')

    for node in head_to_head_adjList.keys():
        sender_name = f"{node[0]} , {node[1]} {node[2]}"
        dot.node(sender_name, color="#ffcccb", style='filled')
        sender_name = f"{node[0]} , {node[1]}"
        dot.node(sender_name, color="#ffcccb", style='filled')

    edges_added = []
    # for every q k v node, plot an edge to output node
    for node in head_to_head_adjList.keys():
        sender_name = f"{node[0]} , {node[1]} {node[2]}"
        receiver_name = f"{node[0]} , {node[1]}"
        dot.edge(sender_name, receiver_name, color = 'red')
        edges_added.append((sender_name, receiver_name))

    for node in mlp_to_head_adjList.keys():
        sender_name = f"{node[0]} , {node[1]} {node[2]}"
        dot.node(sender_name, color="#ffcccb", style='filled')
        sender_name = f"{node[0]} , {node[1]}"
        dot.node(sender_name, color="#ffcccb", style='filled')

    # for every q k v node, plot an edge to output node
    for node in mlp_to_head_adjList.keys():
        sender_name = f"{node[0]} , {node[1]} {node[2]}"
        receiver_name = f"{node[0]} , {node[1]}"
        if (sender_name, receiver_name) not in edges_added:
            dot.edge(sender_name, receiver_name, color = 'red')

    def loop_adjList(adjList):
        for end_node, start_nodes_list in adjList.items():
            if isinstance(end_node, int):
                receiver_name = "MLP " + str(end_node)
            elif isinstance(end_node, tuple):
                if len(end_node) == 3:
                    receiver_name = f"{end_node[0]} , {end_node[1]} {end_node[2]}"
                elif len(end_node) == 2:
                    receiver_name = f"{end_node[0]} , {end_node[1]}"
            else:
                receiver_name = 'resid_post'
            for start in start_nodes_list:
                if isinstance(start, int):
                    sender_name = "MLP " + str(start)
                elif isinstance(start, tuple):
                    if len(start) == 3:
                        sender_name = f"{start[0]} , {start[1]} {start[2]}"
                    elif len(start) == 2:
                        sender_name = f"{start[0]} , {start[1]}"
                dot.node(sender_name, color="#ffcccb", style='filled')
                dot.node(receiver_name, color="#ffcccb", style='filled')
                dot.edge(sender_name, receiver_name, color = 'red')

    loop_adjList(head_to_head_adjList)
    loop_adjList(mlp_to_mlp_adjList)
    loop_adjList(head_to_mlp_adjList)
    loop_adjList(mlp_to_head_adjList)
    loop_adjList(heads_to_resid)
    loop_adjList(mlps_to_resid)

    # Display the graph in Colab
    # display(Source(dot.source))

    # Save the graph to a file
    dot.format = 'png'  # You can change this to 'pdf', 'png', etc. based on your needs
    dot.render(filename)
    files.download(filename + ".png")

In [114]:
plot_graph_adjacency_qkv(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                         mlp_to_head_adjList, heads_to_resid, mlps_to_resid, filename="qkv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## rewrite no qkv fn

In [115]:
def plot_graph_adjacency(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                             mlp_to_head_adjList, heads_to_resid, mlps_to_resid,
                             filename="circuit_graph", highlighted_nodes=None):
    dot = Digraph()
    dot.attr(ranksep='0.45', nodesep='0.11')  # vert height- ranksep, nodesep- w

    def loop_adjList(adjList):
        for end_node, start_nodes_list in adjList.items():
            if isinstance(end_node, int):
                receiver_name = "MLP " + str(end_node)
            elif isinstance(end_node, tuple):
                receiver_name = f"{end_node[0]} , {end_node[1]}"
            else:
                receiver_name = 'resid_post'
            for start in start_nodes_list:
                if isinstance(start, int):
                    sender_name = "MLP " + str(start)
                elif isinstance(start, tuple):
                    sender_name = f"{start[0]} , {start[1]}"
                dot.node(sender_name, color="#ffcccb", style='filled')
                dot.node(receiver_name, color="#ffcccb", style='filled')
                dot.edge(sender_name, receiver_name, color = 'red')

    loop_adjList(head_to_head_adjList)
    loop_adjList(mlp_to_mlp_adjList)
    loop_adjList(head_to_mlp_adjList)
    loop_adjList(mlp_to_head_adjList)
    loop_adjList(heads_to_resid)
    loop_adjList(mlps_to_resid)

    # Save the graph to a file
    dot.format = 'png'  # You can change this to 'pdf', 'png', etc. based on your needs
    dot.render(filename)
    files.download(filename + ".png")

In [116]:
plot_graph_adjacency(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                         mlp_to_head_adjList, heads_to_resid, mlps_to_resid, filename="no qkv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>